# Pneumothorax Localization (Faster R-CNN)

## Data Augmentation using Translate- Crop- and Zoom-transforms

- TODO

In [1]:
from data.dicom import DICOMFolderDataset

valid_data = DICOMFolderDataset('siim-acr-data/dev-pneumothorax/**/**/*.dcm', 'siim-acr-data/train-dev.csv')

## Faster R-CNN Detection Network

In [4]:
import mxnet as mx
from network.rcnn import RoentgenFasterRCNN

ctx = mx.cpu(0)

data = mx.sym.var('data')
im_info = mx.sym.var('im_info')
gtbox = mx.sym.var('gtbox')# change manually to corner format

pneumothorax = RoentgenFasterRCNN(num_classes=2, Nt=0.7, rpn_output=True, scales=[0.25,0.15,0.05], ratios=[2,1,0.5])
pneumothorax.hybridize()

# Kaiming initialization from uniform [-c,c] c = sqrt(2/Nin) 
pneumothorax.collect_params().initialize(mx.init.Xavier(factor_type='in', magnitude=0.44444), ctx=ctx)

bboxes, classes, rpn_cls_prob, rpn_bbox_pred, rpn_label, rpn_bbox_target = pneumothorax(data, im_info, gtbox)

ValueError: not enough values to unpack (expected 6, got 1)

In [6]:
import mxnet as mx
from mxnet import nd

study = 1
ctx = mx.cpu(0)

pneumothorax = RoentgenFasterRCNN(num_classes=2, Nt=0.7, rpn_output=True, scales=[0.25,0.15,0.05], ratios=[2,1,0.5])
pneumothorax.hybridize()

data, label = valid_data.__getitem__(study)
X = nd.array(data.reshape((1, 1, 1024, 1024)), ctx=ctx)
im_info = nd.array([[1024,1024,1]])

bboxes, classes, rpn_cls_prob, rpn_bbox_pred, rpn_label, rpn_bbox_target = pneumothorax(X, im_info, label)

RuntimeError: Parameter 'conv23_weight' has not been initialized. Note that you should initialize parameters and create Trainer with Block.collect_params() instead of Block.params because the later does not include Parameters of nested child Blocks

In [18]:
label# x1>x2 in study 8 format='corner'


[ 83.  15. 304. 183.]
<NDArray 4 @cpu(0)>

## Load RPN Model

In [11]:
from mxnet import gluon

pneumothorax = gluon.nn.SymbolBlock.imports("model/roentgen-anchor-rpn-symbol.json", ['data0','data1','data2'], "model/roentgen-anchor-rpn-0200.params", ctx=ctx)

In [13]:
pneumothorax

SymbolBlock(
<Symbol group [roentgenfasterrcnn0_slice_axis0, proposallayer0_multiproposal0, proposalnetwork0_reshape1, conv22_fwd, anchortargetlayer0_custom0, anchortargetlayer0_custom0]> : 3 -> 6
)

In [34]:
import mxnet as mx
from mxnet import nd

study = 8
ctx = mx.cpu(0)

data, label = valid_data.__getitem__(study)
X = nd.array(data.reshape((1, 1, 1024, 1024)), ctx=ctx)
im_info = nd.array([[1024,1024,1]])

bboxes, classes, rpn_cls_prob, rpn_bbox_pred, rpn_label, rpn_bbox_target = pneumothorax(X, im_info, label)

In [35]:
label


[541. 812. 877. 914.]
<NDArray 4 @cpu(0)>

In [36]:
bboxes


[[47.92309  -8.       68.673096 79.82617 ]]
<NDArray 1x4 @cpu(0)>

## Testing Anchorbox Target Generation

In [24]:
bboxes


[[ 5.126441   2.2516594 25.871796  28.751112 ]]
<NDArray 1x4 @cpu(0)>

In [25]:
rpn_label


[[0.000e+00 1.000e+00 2.000e+00 ... 9.213e+03 9.214e+03 9.215e+03]]
<NDArray 1x9216 @cpu(0)>

In [26]:
rpn_bbox_target


[[[ 0.0000000e+00  0.0000000e+00  2.3821542e-30 ... -1.3696859e-20
    6.9229088e-07  8.5734239e+27]
  [-2.2664384e+16  9.1047469e+05  9.0711414e-18 ... -3.3377890e+22
    1.4439362e-05 -1.5571007e+02]
  [ 4.1543501e-29  1.0390956e+38 -2.1593335e+27 ... -3.0414054e-15
    7.3356980e+23 -1.8611940e+25]
  [-2.9247278e-24  1.2544697e-05  1.2345088e+35 ...  1.2628517e-21
    6.1361636e+33 -3.8426647e+30]]]
<NDArray 1x4x9216 @cpu(0)>

In [27]:
rpn_bbox_pred.shape

(1, 36, 32, 32)

In [28]:
label


[ 83.  15. 304. 183.]
<NDArray 4 @cpu(0)>

## Testing Multi Proposal Block

In [51]:
# testing the output scores of a multi-proposal
from mxnet import nd,gluon
from mxnet.symbol.contrib import Proposal

class MyBlock(gluon.nn.HybridBlock):
    def hybrid_forward(self, F, cls_prob, bbox_pred, im_info):
        proposal = Proposal(cls_prob, bbox_pred, im_info, output_score=True, rpn_min_size=5, rpn_post_nms_top_n=4, scales=[0.05], ratios=[0.01])
        return proposal

mytest = MyBlock()
mytest.hybridize()

In [52]:
cls_scores = nd.array([[
    [[0.0,0.0,0.0],[0.0,-500,0.0],[0.0,-100,0.0]],
    [[0.0,0.0,0.0],[0.0,500,0.0],[0.0,100,0.0]]
    ]])
bbox_pred = nd.zeros((1,4,3,3))
im_info = nd.array([[15,15,1]])

In [53]:
cls_scores


[[[[   0.    0.    0.]
   [   0. -500.    0.]
   [   0. -100.    0.]]

  [[   0.    0.    0.]
   [   0.  500.    0.]
   [   0.  100.    0.]]]]
<NDArray 1x2x3x3 @cpu(0)>

In [54]:
nd.softmax(cls_scores,axis=1)


[[[[0.5 0.5 0.5]
   [0.5 0.  0.5]
   [0.5 0.  0.5]]

  [[0.5 0.5 0.5]
   [0.5 1.  0.5]
   [0.5 1.  0.5]]]]
<NDArray 1x2x3x3 @cpu(0)>

In [55]:
bbox_pred


[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]]
<NDArray 1x4x3x3 @cpu(0)>

In [235]:
im_info


[[15. 15.  1.]]
<NDArray 1x3 @cpu(0)>

In [236]:
# why are the output scores always -1
mytest(cls_scores, bbox_pred, im_info)

[
 [[ 0.    1.5   5.45 13.5   9.55]
  [ 0.   11.5   5.45 16.5   9.55]
  [ 0.    1.5  11.5  13.5  16.5 ]
  [ 0.   11.5  11.5  16.5  16.5 ]]
 <NDArray 4x5 @cpu(0)>,
 
 [[-1.]
  [-1.]
  [-1.]
  [-1.]]
 <NDArray 4x1 @cpu(0)>]